<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/PassaggichiaveMKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Tabella dei Dati Essenziali e Costanti del Progetto**

| Parametro / Dato | Valore | Unità | Fonte / Scopo | Note |
| :--- | :--- | :--- | :--- | :--- |
| **PARAMETRI FISICI & GEOMETRICI (Ground Truth)** | | | | |
| Diametro Nominale Foro (`D_nom`) | **6.0** | mm | Tesi / **Calibrazione** | La nostra "stele di Rosetta". È la misura reale che useremo per calibrare ogni patch. |
| Area Nominale Foro (`A_nom`) | **28.274** | mm² | Calcolata (`π*r²`) | Serve per calcolare `Adel` dai dati `Atot` umani e per lo `Shape Factor`. |
| Spessore Laminato | ~2.0 | mm | Tesi | Non usato nei modelli attuali, ma fondamentale per future simulazioni FEM. |
| **PARAMETRI DELLA SCANSIONE ORIGINALE** | | | | |
| Risoluzione Scansione | **600** | DPI | Tesi / Specifiche | Utilizzata per il calcolo della scala teorica. |
| Scala Teorica Originale | **23.62** | pixel/mm | Calcolata (`600/25.4`) | **Valida SOLO sulla scansione intera, non sulle patch ridimensionate.** |
| Diametro Foro in Pixel (Teorico) | **~142** | pixel | Calcolato (`6 * 23.62`) | Utile come controllo di sanità sui primi script di rilevamento. |
| **PARAMETRI DELLA PIPELINE ATTUALE (YOLO -> UNet)** | | | | |
| Dimensione Patch di Input (UNet) | **512 x 512** | pixel | Tuo script di crop | **Questo ridimensionamento ha causato la perdita della scala originale.** |
| **Scala Empirica Misurata su Patch** | **~35.8** | pixel/mm | Nostra misura manuale | **Valore indicativo** che conferma la discrepanza e la necessità di una calibrazione per patch. |
| **CLASSI NELLE MASCHERE UNet** | | | | |
| Sfondo | **0** | (valore pixel) | UNet script | Classe da ignorare nei calcoli. |
| Foro | **1** | (valore pixel) | UNet script | Maschera del foro. La UNet la separa dal danno. |
| Delaminazione | **2** | (valore pixel) | UNet script | Maschera del danno. **È questa che dobbiamo misurare.** |

---

### **Confronto Dati: Umano vs. Nostro (Il Problema da Risolvere)**

Questa tabella evidenzia la discrepanza che dobbiamo risolvere. Prendo il Foro 1 come esempio lampante:

| Metrica | Dato Umano (Tesi) | Dato Nostro (CSV sballato) | Stato |
| :--- | :--- | :--- | :--- |
| **Adel** | **3.08 mm²** | **6.14 mm²** | 🔴 **Sbagliato!** (Errore del ~100%) |
| **DMAX** | **6.22 mm** | **15.74 mm** | 🔴 **Sbagliato!** (Errore del ~150%) |
| **Forza** | **89.50 N** | (da predire) | 🔜 Obiettivo Futuro |

### **Cosa Facciamo Adesso (Il Piano Pratico Immediato)**

Come hai detto tu, lavoriamo ripartendo dalle patch e dalle maschere che hai già.

1.  **Azione #1: Calibrazione Automatica (ORA).**
    *   Prendiamo la cartella con le tue **patch radiografiche** (`.jpg`, 512x512).
    *   Eseguiamo lo **script di calibrazione automatica** che ti ho fornito. Questo script misurerà il diametro del foro in pixel in ogni patch e creerà il file `calibrazione_scale_patch.csv`.
    *   **Risultato:** Un file che associa ogni patch alla sua scala `px/mm` corretta.

2.  **Azione #2: Estrazione Features Corretta.**
    *   Modifichiamo lo script di estrazione features. Per ogni maschera:
        *   Legge il nome del file (es. `H001_..._pred.png`).
        *   Cerca la scala per `H001` nel file `calibrazione_scale_patch.csv`.
        *   Misura area e DMAX in pixel dalla maschera.
        *   **Usa la scala corretta per convertire i valori in mm e mm².**
    *   **Risultato:** Un nuovo file `features_corrette_e_verificate.csv` con dati finalmente confrontabili con quelli del tesista.

Iniziamo con l'**Azione #1**. Prepara la cartella con le patch `.jpg` e il codice di calibrazione che ti ho passato. È il passo più importante per sbloccare tutto il resto.


#### **Fase 1: La Scansione Originale (Il Mondo Reale in Pixel)**

*   **Immagine di Riferimento:** `T0_90_1A_ingresso.jpg` (e le altre scansioni complete).
*   **Scala Teorica (DPI):** 600 DPI (dots per inch).
    *   Conversione: 1 pollice = 25.4 mm
    *   **Scala Originale (px/mm):** `600 / 25.4 = **23.62 pixel per mm**`.
*   **Dimensione Fisica Riferimento:** Diametro nominale del foro `D_nom = **6.0 mm**`.
*   **Dimensione in Pixel (Teorica):** Sulla scansione originale, un foro da 6 mm dovrebbe avere un diametro di `6.0 mm * 23.62 px/mm ≈ **142 pixel**`.

#### **Fase 2: Dal Rilevamento YOLO al Ritaglio della Patch**

*   **Script di Riferimento:** `yolorilevazionefori.ipynb` e gli script successivi di ordinamento e crop.
*   **Processo:**
    1.  YOLO rileva i fori sulla scansione intera (`imgsz=1280` o `(H,W)` nativa).
    2.  Le coordinate delle bounding box vengono salvate e ordinate con K-Means (l'ordinamento bustrofedico).
    3.  Uno script di crop prende il centro `(cx, cy)` di ogni foro rilevato.
    4.  Viene ritagliata una patch quadrata attorno a quel centro. Hai usato `HS = 350` (patch 700x700) e `HS = 256` (patch 512x512).
    5.  **IL PASSAGGIO CRUCIALE:** In uno degli script, c'è `patch = cv2.resize(patch, (TARGET_SIZE, TARGET_SIZE), ...)` dove `TARGET_SIZE` è `512`.

#### **Fase 3: La Patch Radiografica (L'Input per la UNet)**

*   **Immagine di Riferimento:** L'immagine della patch singola che mi hai mostrato.
*   **Dimensione:** **512x512 pixel**.
*   **Il Problema:** Questa immagine è il risultato del `resize` della Fase 2. La sua scala in `pixel/mm` **NON è più 23.62**. È una nuova scala che dobbiamo calcolare.

#### **Fase 4: La Maschera Predetta (L'Output della UNet)**

*   **Immagine di Riferimento:** L'immagine della maschera in bianco e nero che mi hai mostrato.
*   **Dimensione:** **512x512 pixel** (la stessa della patch di input).
*   **Contenuto:** I valori dei pixel rappresentano le classi (es. 0=Sfondo, 1=Foro, 2=Danno).
*   **Obiettivo:** Misurare l'area e il DMAX del danno (pixel di classe 2) su questa maschera e convertirli in `mm²` e `mm` usando la scala corretta.


In [ ]:
### **Script di Calibrazione Automatica della Scala**
'''
Questo script fa una sola cosa, ma la fa bene:
1.  Legge ogni immagine `.jpg` da una cartella di input (le tue patch radiografiche).
2.  Usa l'algoritmo di visione artificiale `HoughCircles` per trovare il foro da 6 mm.
3.  Calcola la scala `pixel/mm` per quella specifica immagine.
4.  Salva tutti i risultati in un file CSV chiamato `calibrazione_scale_patch.csv`.

```python
'''# =============================================================================
# SCRIPT DI CALIBRAZIONE AUTOMATICA DELLA SCALA PER OGNI PATCH
# =============================================================================

# --- 1. INSTALLAZIONI E IMPORT NECESSARI ---
# (Esegui questa cella una sola volta all'inizio)
!pip install -q opencv-python pandas matplotlib
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from google.colab import files # Per scaricare il file CSV alla fine

# --- 2. IMPOSTAZIONI (LE UNICHE COSE DA MODIFICARE) ---

# Diametro reale del foro in millimetri. Questa è la nostra "verità" per la calibrazione.
DIAMETRO_REALE_FORO_MM = 6.0

# Imposta il percorso alla cartella che contiene le TUE patch radiografiche (.jpg).
# Esempio per Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# RADIO_PATCHES_DIR = Path("/content/drive/MyDrive/PATCHES_RADIO_Dataset")

# Per un test iniziale, puoi caricare le immagini manualmente in una cartella in Colab.
RADIO_PATCHES_DIR = Path("/content/drive/MyDrive/PATCHES_RADIO_Dataset")#"/content/immagini_da_calibrare")
RADIO_PATCHES_DIR.mkdir(exist_ok=True) # Crea la cartella se non esiste
print(f"ATTENZIONE: Lo script cercherà le immagini in '{RADIO_PATCHES_DIR}'")
print("Carica le tue patch .jpg in quella cartella.")


# --- 3. FUNZIONE DI CALIBRAZIONE ---

def get_scale_from_radiograph(radiograph_path: Path, visualize=False):
    """
    Carica un'immagine radiografica, trova il foro centrale e calcola la scala pixel/mm.
    """
    if not radiograph_path.exists():
        print(f"Attenzione: Immagine non trovata a {radiograph_path}")
        return None, None

    # Carica l'immagine in scala di grigi
    image = cv2.imread(str(radiograph_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None, None

    # Applica un leggero blur per ridurre il rumore e aiutare l'algoritmo
    blurred_image = cv2.GaussianBlur(image, (9, 9), 2)

    # Parametri per l'algoritmo HoughCircles. Potrebbero richiedere aggiustamenti.
    # dp: Rapporto inverso della risoluzione dell'accumulatore. 1.2 è un buon punto di partenza.
    # minDist: Distanza minima tra i centri dei cerchi. Mettiamola grande per trovare solo il foro principale.
    # param1: Soglia superiore per l'edge detector di Canny interno.
    # param2: Soglia per il centro del cerchio. Più è basso, più "falsi" cerchi trova.
    # minRadius, maxRadius: Limiti per il raggio del cerchio. FONDAMENTALI!

    # Stimiamo un range ragionevole per il raggio basandoci sulle dimensioni dell'immagine
    img_height, img_width = image.shape
    min_r = int(img_height * 0.15) # min raggio 15% dell'altezza
    max_r = int(img_height * 0.40) # max raggio 40% dell'altezza (più flessibile)

    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1.2, minDist=img_width,
                               param1=70, param2=45, minRadius=min_r, maxRadius=max_r)

    if circles is not None:
        # Trovato almeno un cerchio!
        circles = np.round(circles[0, :]).astype("int")
        (x, y, r) = circles[0] # Prendiamo il primo (e unico) cerchio trovato

        diametro_pixel = r * 2
        scala_px_per_mm = diametro_pixel / DIAMETRO_REALE_FORO_MM

        if visualize:
            output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            cv2.circle(output_image, (x, y), r, (0, 255, 0), 4) # Cerchio verde
            cv2.circle(output_image, (x, y), 5, (0, 0, 255), -1) # Punto rosso al centro
            testo = f"Scala: {scala_px_per_mm:.2f} px/mm"
            cv2.putText(output_image, testo, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return scala_px_per_mm, output_image

        return scala_px_per_mm, None
    else:
        # Nessun cerchio trovato automaticamente
        return None, image if visualize else None

# --- 4. BLOCCO DI ESECUZIONE ---

# Prendi tutte le immagini .jpg nella cartella
all_radio_paths = sorted(list(RADIO_PATCHES_DIR.glob("*.jpg")))
if not all_radio_paths:
    all_radio_paths = sorted(list(RADIO_PATCHES_DIR.glob("*.png"))) # Prova anche i png

if not all_radio_paths:
    print(f"❌ ERRORE: Nessuna immagine .jpg o .png trovata nella cartella '{RADIO_PATCHES_DIR}'.")
    print("Assicurati di aver caricato le immagini prima di eseguire questa cella.")
else:
    print(f"Trovate {len(all_radio_paths)} patch. Inizio calibrazione...")

    # Visualizziamo un esempio per un controllo di qualità
    print("\n--- TEST DI VISUALIZZAZIONE SULLA PRIMA IMMAGINE ---")
    scala_test, img_test = get_scale_from_radiograph(all_radio_paths[0], visualize=True)
    if scala_test:
        print(f"Scala calcolata per '{all_radio_paths[0].name}': {scala_test:.2f} px/mm")
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB))
        plt.title("Visualizzazione Rilevamento Cerchio")
        plt.axis('off')
        plt.show()
    else:
        print("Rilevamento fallito sull'immagine di test. Prova a modificare i parametri di HoughCircles (param2, minRadius, maxRadius).")

    # Eseguiamolo su tutte le immagini per creare il nostro file di calibrazione
    print("\n--- CALIBRAZIONE DI MASSA SULL'INTERO DATASET ---")
    calibration_data = []
    for path in tqdm(all_radio_paths, desc="Calibrazione Immagini"):
        scala, _ = get_scale_from_radiograph(path, visualize=False)

        entry = {'filename': path.name}
        if scala:
            entry['scala_px_per_mm'] = scala
        else:
            entry['scala_px_per_mm'] = np.nan # Mettiamo NaN se il rilevamento fallisce
        calibration_data.append(entry)

    # Converti i risultati in un DataFrame di Pandas
    df_calibrazione = pd.DataFrame(calibration_data)

    output_filename = "calibrazione_scale_patch.csv"
    df_calibrazione.to_csv(output_filename, index=False)

    print("\n--- RISULTATI CALIBRAZIONE ---")
    print(df_calibrazione.head())
    print(f"\n✅ File di calibrazione '{output_filename}' creato con successo!")

    # Controlliamo se ci sono stati fallimenti
    fallimenti = df_calibrazione['scala_px_per_mm'].isna().sum()
    if fallimenti > 0:
        print(f"\n⚠️ ATTENZIONE: {fallimenti} immagini non sono state calibrate correttamente.")
    else:
        print("\n🎉 OTTIMO: Tutte le immagini sono state calibrate con successo.")

    # Analizziamo la distribuzione delle scale
    plt.figure(figsize=(10, 5))
    df_calibrazione['scala_px_per_mm'].hist(bins=30, edgecolor='black')
    plt.title("Distribuzione delle Scale Calcolate (px/mm)")
    plt.xlabel("Scala (pixel/mm)")
    plt.ylabel("Numero di Patch")
    plt.show()

    # Offri il download del file CSV
    files.download(output_filename)